In [11]:
import sys
sys.path.append("./MNE-Python tutorial")

In [109]:
from bnci import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

from mne.filter import filter_data

In [105]:
SFREQ = 250 #250Hz
LOWCUT = 4
HIGHCUT = 40

LEARNING_RATE = 0.001
BATCH_SIZE = 16
EPOCHS = 10

데이터 준비

In [106]:
#Training Data
t_datas= []
t_labels = []
for i in range(1,10):
    data, label = get_data_2a(i, True)
    # bandpass filter
    data = np.array([filter_data(data=d, sfreq=SFREQ, l_freq=LOWCUT, h_freq=HIGHCUT,
                                        verbose=False, fir_design='firwin') for d in data])
    # crop the data (imagery part -- 2s to 6s)
    data = data[:, :, int(SFREQ * 2):SFREQ * 6]

    t_datas.append(data)
    t_labels.append(label)

#Evaluation Data
e_datas=[]
e_labels=[]
for i in range(1,10):
    data, label = get_data_2a(i, False)
    # bandpass filter
    data = np.array([filter_data(data=d, sfreq=SFREQ, l_freq=LOWCUT, h_freq=HIGHCUT,
                                        verbose=False, fir_design='firwin') for d in data])
    # crop the data (imagery part -- 2s to 6s)
    data = data[:, :, int(SFREQ * 2):SFREQ * 6]

    e_datas.append(data)
    e_labels.append(label)    

https://tutorials.pytorch.kr/beginner/basics/buildmodel_tutorial.html  
https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html  
참조

In [85]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


In [107]:
class ShallowConvNet(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.tempConv = nn.Conv2d(1, 40, (1,25))
        self.spaConv = nn.Conv2d(40, 40, (22,1)) 

        # Batch Normalization\n",
        self.batch_norm = nn.BatchNorm2d(40, momentum=0.1, affine=True, eps=1e-5)

        self.pool = nn.AvgPool2d((1,75), (1,15))
        self.dropout1 = nn.Dropout(p=0.3)
        self.flatten = nn.Flatten()

        # Linear Layer for Classification
        self.fc = nn.Linear(2440, 4)
    
    def forward(self, x):
        # Temporal Convolution
        x = self.tempConv(x)
        x = self.spaConv(x)
        x = self.batch_norm(x)
        x = torch.square(x)
        # Mean Pooling
        x = self.pool(x)
        x = torch.log(torch.clamp(x,min=1e-6))
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        
        # Linear Classification
        x = self.fc(x)
        
        return F.softmax(x, dim=1)
    
net = ShallowConvNet()
net.to(device) #using gpu


ShallowConvNet(
  (tempConv): Conv2d(1, 40, kernel_size=(1, 25), stride=(1, 1))
  (spaConv): Conv2d(40, 40, kernel_size=(22, 1), stride=(1, 1))
  (batch_norm): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 75), stride=(1, 15), padding=0)
  (dropout1): Dropout(p=0.3, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=2440, out_features=4, bias=True)
)

In [97]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [99]:
for epoch in range(EPOCHS):
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for _, (data,labels) in enumerate(train_loader):
        ##need to implement train_loader
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        outputs = net(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        
        loss.backward()  # Backward pass (compute gradients)
        optimizer.step()  # Update weights
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")

print("Training complete.")

RuntimeError: Calculated padded input size per channel: (1875 x 22). Kernel size: (1 x 25). Kernel size can't be greater than actual input size

In [95]:
evaluation_epochs_arrays = []
evaluation_classes = []
for i in range(1,10):
    res_data, res_class = get_data_2a(i, False)
    epochs_array = EEG_to_epochs(res_data, res_class,sfreq = 250,)
    evaluation_epochs_arrays.append(epochs_array)
    evaluation_classes.append(res_class)

ev_concat_epochs = mne.concatenate_epochs(evaluation_epochs_arrays)
ev_concat_labels = np.concatenate(evaluation_classes)

ev_dataset = EEGDataset(ev_concat_epochs, ev_concat_labels)
dataloader = DataLoader(ev_dataset, batch_size=32, shuffle=True)

(288, 22, 1875)
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
(288, 22, 1875)
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
(288, 22, 1875)
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
(288, 22, 1875)
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
(288, 22, 1875)
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
(288, 22, 1875)
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
(288, 22, 1875)
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
(288, 22, 1875)
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
(288, 22, 1875)


In [84]:
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
    
        # 신경망에 이미지를 통과시켜 출력을 계산합니다
        outputs = net(inputs)
        
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 67 %
